# Bernstein-Vazirani Algorithm

#### Problem Statement:

- Given an oracle function $f:\{0,1\}^n \rightarrow \{0,1\}$, defined as $ f(\mathbf{x}) = x \cdot s $, find the secret bit string $s$.

**Note:**

- $x \cdot s$ represents the inner product of the bit strings modulo $2$.

- The size of the bitstring should be the same as the size of input $x$ for the operation $x.s$ to make sense.

- The unitary operator representing the oracle now takes the form:  
$$ U_f: \ket{x}\ket{y} \mapsto \ket{x}\ket{y \oplus ( x \cdot s)} $$


#### <font color='red'>Exercise:</font>

Find $s$ for the function that provides the below outputs: (here $n=2$)
\begin{align*}
	f(00)&=0\\
	f(01)&=1\\
	f(10)&=1\\
	f(11)&=0\\
\end{align*}

Example: Say $x=10$, from above $f(x)=x.s= 1$
$$ 1= x.s= (x_1x_2).(s_1s_2)= x_1s_1 + x_2s_2 (mod 2)= 1s_1 + 0s_2 (mod 2)= s_1$$
Thus $s_1= 1$, similarly find $s_2$ to get the full string $s_1s_2$.

Note that you would require $2$ calls to the function to find $s$ completely.

#### <font color='red'> **Exercise:** *Classical Complexity-* Clasically, how many oracle calls you need to make to find the secret string? </font>

        (your answer here)

#### **Quantum Algorithm:** 

We construct a $n+1$ qubit quantum circuit.

- Set the ($n+1$)th qubit to state $\ket{-}$ by applying $X$ and $H$ gates.
- Apply $H$ to first $n$ qubits.
- Apply $U_f$. (*the oracle*: required to be given to the algorithm)
- Apply $H$ to first $n$ qubits.
- Measure the first $n$ qubits to obtain $s$.

This is exactly the same algorithm as the Deutsch-Jozsa which we covered in a previous lab material. Only the oracle has a slighly modified form. 

![DeutschjoZSACircuit](Images/DJ.png)

#### <font color='red'>Exercise: Work out the stages of the algorithm looking at the circuit above and check whether you get the below states: </font>

$$ \ket{\psi_0} = \ket{0}^{\otimes n} \ket{0}$$

$$ \ket{\psi_1} = \ket{0}^{\otimes n} \ket{1}$$

$$ \ket{\psi_2} = \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2^n-1} \ket{x} \otimes \ket{-} $$

$$ \ket{\psi_3} = \bigg[ \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} (-1)^{x.s} \ket{x} \bigg] \otimes \ket{-} $$


- Remember from the $PS(07)$ notebook, $ H^{\otimes n} \ket{x} = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} (-1)^{x \cdot z} \ket{z}. $

- We also know that the $H^{\otimes n}$ operator is its own inverse. 

- Thus, $H^{\otimes n}\ket{a} = \ket{b} \Longleftrightarrow H^{\otimes n}\ket{b} = \ket{a}$. 

So, the first $n$ qubits state of $\ket{\psi_3}$ is the state obtained after applying $H^{\otimes n}$ to $\ket{s}$.

Thus applying $H^{\otimes n}$ to the input qubits (the first $n$ qubits), should provide us:

$$ \ket{\psi_4} = \ket{s} \otimes \ket{-} $$

Measurement of the first $n$ qubits will provide string $s$ with probability $1$.

##### *Note that the Bernstein-Vazirani problem was artificially created to demonstrate the advantage of a quantum algorithm over classical algorithms.*

##### To implement the algorithm, we will first design some specific oracles and then use the algorithm to determine the string $s$ encoded in the oracle.

#### <font color= 'red'> Lab Exercise: Design the oracles given below </font> 
(here n=4 , hence we would require a Quantum Circuit with 5 qubits)

- Given $\textbf{s} = 0110$, implement a function that returns an oracle for the function  $ f(\mathbf{x}) = \mathbf{x} \cdot \mathbf{s} $. *(oracle1, done for you below)*


- Given $\textbf{s} = 1001$, implement a function that returns an oracle for the function  $ f(\mathbf{x}) = \mathbf{x} \cdot \mathbf{s} $. *(oracle2, you need to do)*

Can you generalize the oracle construction for any bitstring $s$, irrespective of its size.



In [6]:
import qiskit
from qiskit import QuantumCircuit

def oracle1():  #for s=0110
    circuit = QuantumCircuit(5)
    circuit.barrier()

    circuit.cx(1, 4)
    circuit.cx(2, 4)

    circuit.barrier()
    return circuit


def oracle2(): #for s= 1001
    circuit = QuantumCircuit(5)
    circuit.barrier()

    #your code here

    circuit.barrier()
    return circuit

In [14]:
#Chosing a random oracle
import random
# Create the list of oracles
oracle_list = [oracle1, oracle2]

# Randomly select an index using randrange(4)
random_index = random.randrange(2)

print(random_index)

# Call the selected oracle
selected_oracle = oracle_list[random_index]

selected_oracle()

0


##### Given the oracle function `selected_oracle()` for $f$,construct a circuit that implements the Bernstein-Vazirani algorithm described above to find out $s$.

In [15]:
from qiskit import QuantumCircuit, execute, Aer

n=4 #(size of the string or the input 'x')

#Initialization
bv_circuit = QuantumCircuit(n+1, n)  #measurement only on the first n qubits, hence n classical registers would suffice
bv_circuit.x(n)
bv_circuit.h(range(n+1))

#Apply oracle
bv_circuit.compose(selected_oracle(), inplace=True)

#Apply Hadamard to all qubits and measure the first n qubits
bv_circuit.h(range(n))
bv_circuit.measure(range(n), range(n))

#Draw the circuit, if needed

#bv_circuit.draw(output="mpl")

#extracting the results of measurement to determine 's'
job = execute(bv_circuit, Aer.get_backend('qasm_simulator'),shots=10000)
counts = job.result().get_counts()
for outcome in counts:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome,"is observed",counts[outcome],"times")

0110 is observed 10000 times


Since a classical computation of a requires $n$ function calls, we have obtained a “quantum speedup” of $n$ (polynomial speedup). 

The procedure is analagous to Deutsch-Jozsa algorithm. The first set of Hadamards generates a superposition of inputs to the oracle $Uf$ which evaluates the function for all $2^{n}$ inputs using *quantum parallelism*, and then the second set of Hadamards destroys all the outputs apart from $s$, using *quantum interference*.


#### <font color= 'red'> Homework: </font> 

- Given $\textbf{s} = 0110101$, implement a function that returns an oracle for the function  $ f(\mathbf{x}) = \mathbf{x} \cdot \mathbf{s} $.
- Implement the Berstein-Vazirani algorithm for the above oracle and check if you obtain the required string.
- What is the circuit width and depth used for solving this specific problem?


# Simon's Algorithm

#### Problem Statement:

- Given an oracle function $f: \{0,1\}^{n} \rightarrow \{0,1\}^{n}$, such that $f(x) = f(x\oplus s)$, the goal is to determine the bitstring $s$ (where $s$ $\in$ {0,1}$^{n}$ and $s\ne 0^{n}$).


**Note:**

- Unlike the previous problem, here we don't know the functional form of $f$.
- The secret string $s$ can be considered as a "mask" which hides the underlying encoding ($f$).
- As long as $s$ is not the zero bitstring, the function is two-to-one i.e. mapping two elements from the domain to one value from the range.

<i>Simon's problem can also be defined as the problem of determining whether $f$ is two-to-one or one-to-one, in which case one needs to determine whether $s$ is a zero string or not.</i>

#### <font color= 'red'> Exercise: </font> 

- For two inputs $x_1$ and $x_2$ for which $f(x_1) = f(x_2)$, show that $x_1⊕x_2$ yields the secret string $s$, given that the function $f$ satisfies the property $f(x)=f(x\oplus s)$.
- What is the classical complexity for the Simon's problem? (look at the worst-case scenario)

#### Have you hear of the Birthday problem?

- In a set of $n$ randomly chosen people, what should be the value of $n$ such that the probability of at least two people sharing a birthday exceeds 50%?

You can read more about this problem [here](https://betterexplained.com/articles/understanding-the-birthday-paradox/).